## Trial 1

In [350]:
import requests
import pandas
import folium
#####################################
#Europe: 228083900, 380 ft, Oil Tanker
#Third Wish: 366903130
#CSCL Globe: 477712400, 400 m
#Barzan: 229930000, 400 m 
values = ['367526690',
          '367527690',
          '366952890',
          '366903130',
          '477712400',
          '229930000',
        ]

#####################################
#note that geopandas only works in no-arc environments
def tracker(mmsi):
    response = requests.get('https://www.myshiptracking.com/vessels/mmsi-'+ mmsi)
    if response.status_code == 200:
        #print("good")
        trimmed = re.sub("</*strong>|</p><p>|</*small>", "", response.text)
        all_data = str(trimmed.partition("The current position of ")[2]).partition('In this page')[0]
        shipname = str(all_data.partition(" is in ")[0])
        coordinates = re.findall("-?[0-9]{0,3}\.[0-9]{5}", all_data)
        time = re.search("20..-..-........",all_data)
        speed = re.search("[0-9]+.Knots", all_data)
        return ({'Ship Name': shipname ,
                "Latitude" : coordinates[0],
                "Longitude": coordinates[1],
                "Time": time.group(),
                 "Speed": speed.group()});
    else:
        #print("Invalid")
        return "Invalid"

df = pandas.DataFrame(columns=['Ship Name',"Latitude","Longitude","Time","Speed"])
count = 1
for each in values:
    data = tracker(each)
    df1 = pandas.DataFrame(data,columns=['Ship Name',"Latitude","Longitude","Time","Speed"], index = [count])
    df = df.append(df1)
    count = count+1
print(df)

m = folium.Map([40, -90], tiles = 'CartoDB Positron', zoom_start=4)
for index, row in df.iterrows():
    folium.Marker(([row['Latitude'], row['Longitude']]), 
                  popup=row['Ship Name'],
                  icon=folium.Icon(color = 'blue', icon_color = 'white', icon='ship', prefix='fa')
                  ).add_to(m)
m.save('New.html')

           Ship Name  Latitude   Longitude              Time    Speed
1       CONSTITUTION  32.61174   -91.15285  2016-04-13 06:09  9 Knots
2  SEACOR   CHAMPION  29.06957   -90.11900  2019-06-03 01:46  4 Knots
3  SPIRIT OF AMERICA  40.64183   -74.07077  2021-11-04 03:21  0 Knots
4         THIRD WISH  47.67990  -122.40904  2021-10-31 23:26  0 Knots
5         CSCL GLOBE  37.95539    23.58001  2021-11-04 03:18  0 Knots
6             BARZAN  22.40082   114.53896  2021-11-03 13:07  1 Knots


## Trial 1.2

In [3]:
####### Import
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import requests
import pandas
import folium
values = [
    '367526690',#      
    '367527690',#
    '366952890',#
    '366903130',#
    '477712400',#
    '229930000',#
]
          
####### Scraper
def new(mmsi):
    response = requests.get('https://www.myshiptracking.com/vessels/mmsi-'+ mmsi)
    if response.status_code == 200:
        html= response.text
        soup = (BeautifulSoup(html, "html.parser", parse_only= SoupStrainer("strong")))
        blanklist = []
        for string in soup.strings:
            blanklist.append(string)
    return({"Name":blanklist[0],"Coordinates":blanklist[4], "Time":blanklist[5],"Speed":blanklist[6]})

####### Framing
dq = pandas.DataFrame(columns=['Name',"Coordinates",'Time',"Speed"])
count = 1
for each in values:
    dq1 = pandas.DataFrame(new(each),columns=['Name',"Coordinates","Time","Speed"], index = [count])
    dq = dq.append(dq1)
    count += 1
dq[['Lat','Long']] = dq.Coordinates.str.split("/",expand=True,)
dq['Lat'] = dq['Lat'].str.replace(r'°', '')
dq['Long'] = dq['Long'].str.replace(r'°', '')

####### Mapping
m = folium.Map([40, -90], tiles = 'CartoDB Positron', zoom_start=4)
for index, row in dq.iterrows():
    folium.Marker(([row['Lat'], row['Long']]), 
                  popup=row['Name'],
                  icon=folium.Icon(color = 'blue', icon_color = 'white', icon='ship', prefix='fa')
                  ).add_to(m)
m.save('New.html')
print(dq)

                Name              Coordinates              Time       Speed  \
1       CONSTITUTION   32.61174° / -91.15285°  2016-04-13 06:09   0.9 Knots   
2  SEACOR   CHAMPION   29.06957° / -90.11900°  2019-06-03 01:46   5.4 Knots   
3  SPIRIT OF AMERICA   40.64181° / -74.07073°  2021-11-05 21:51     0 Knots   
4         THIRD WISH  47.67990° / -122.40904°  2021-10-31 23:26     0 Knots   
5         CSCL GLOBE    36.39495° / 23.14179°  2021-11-05 21:52  20.4 Knots   
6             BARZAN   22.14439° / 114.45100°  2021-11-05 04:19  15.8 Knots   

         Lat         Long  
1  32.61174     -91.15285  
2  29.06957     -90.11900  
3  40.64181     -74.07073  
4  47.67990    -122.40904  
5  36.39495      23.14179  
6  22.14439     114.45100  
